## Possible solution to assignment 2 - collocation extraction

I'm only going to be using ```re``` and ```string``` from the Python base libraries.

I'll also be using ```numpy``` later for caculating a logarithm.

In [50]:
import re, string
import numpy as np

I'm only going to work with one text here; I won't go into how to iterate but it should be clear from the logic of the programme.

In [51]:
with open("../data/100_english_novels/corpus/Dickens_Expectations_1861.txt", "r", encoding="utf-8") as file:
    text = file.read()

I create a quick tokenizer using ```regex``` which splits on non-alphanumeric characters

In [52]:
# A quick regex tokenizer for splitting files below
def tokenize(input_string):
    tokenizer = re.compile(r"\W+")
    return tokenizer.split(input_string)

I then define my keyword and window size.

In [53]:
keyword = "love"
window_size = 5

We do not need all of our variables to calculate MI, only O11, R1, C1, and N.

What are these variables? Well, in regular languge:

- O11 = Number of times target word (u) appears with collocate (v) within the chosen window size
- C1 = The number of times collocate appears across the whole text/corpus
- R1 = The number of times target word (u) appears with any collocate within a chosen window size <br>

So:
- R1 = total number of all collocates with u
- C1 = total occurrence of collocate v
- N = total length of text/corpus

__Tokenize text__

I'm first going to tokenize the text, make it all lowercase, remove all punctuation, and create a list of tokens.

Why do I do these steps?

In [54]:
# ... tokenize the text and remove punctuation
tokenized_text = []

for word in tokenize(text):
    # Lowercase
    lowercase = word.lower()
    # cleanup punctuation etc
    cleaned = lowercase.translate(str.maketrans('', '', string.punctuation))
    tokenized_text.append(cleaned)

I then create a new list called ```tmp``` (for temporary). And then for every token in the list of tokenized text, I check if that token is equal to my keyword. 

In other words, for any token, is that token the word "love"?

If so, I take 5 words before my target, and 5 words after and append these to my list called ```tmp```.

(The logic here is the same as the KWIC function we made in class)

In [55]:
# create temporary list
tmp = []
# For the target word... 
for idx,word in enumerate(tokenized_text):
    # If it's the keyword...
    if word == keyword:
        # Left context catch start of list
        left_context = max(0, idx-window_size)
        right_context = idx+window_size
        # ... extract all words ± 5 and add to tmp list.
        full_context = tokenized_text[left_context:idx] + tokenized_text[idx+1:right_context]
        # append to list
        tmp.append(full_context)

We now have a list of lists, where each sublist is the context around each specific instance of our target word.

We want to flatten this into one big list of words. So for each sublist in our list ```tmp```, we append each token in the sublist to the variable ```flattened_list```.

In [56]:
# Flatten list
flattened_list = []
# For each sublist in list of lists
for sublist in tmp:
    # For each item in sublist
    for item in sublist:
        # append
        flattened_list.append(item)

We then want to count how often each word appears in the ```flattened_list```. In other words, how often does each word appear in the context of our keyword?

To do this, we create a ```set()``` of words from the flattened list and the use ```.count()``` how often each word in the set appears in ```flattened_list```

In [57]:
# Create a list of collocate counts
collocate_counts = []

# for every collocate 
for word in set(flattened_list):
    # Count how often each word appears as a collocate
    count = flattened_list.count(word)
    # Append tuple of word and count to list
    collocate_counts.append((word, count))

We can then calculate some of our values for the MI formula.

We can calulate R1 because that is how often any collocate occurs with our keyword.

Hence, R1 is just the length of the ```flattened_list``` above! Similarly, N is equal to the size of the full text, or the lenght of ```tokenized_text``` above.

In [58]:
# R1 is how often u appears with any v at all
R1 = len(flattened_list)

# N is the length of the document
N = len(tokenized_text)

We need to calculate the rest of the values individually for each collocate.

```collocate_counts``` contains tuples of ```(collocate, count)``` for every collocate for our keyword.

Hence, the ```count``` value from the tuple is how often the collocate occurs with our keyword, which is how we defined O11!

Lastly, C1 is a count of how often the collocate appears across the dataset in total, either with the keyword (O11) or without the keyword (O22).

In [59]:
# values for calculating MI
frequency_tuples = []

# For every collocate
for collocate, count in collocate_counts:
    # O11 is how often it appears as collocate
    O11 = count
    # R1 is R1
    R1 = R1
    # N is N
    N = N
    # C1 is collocate distribution across the corpus
    C1 = tokenized_text.count(collocate)
    #
    frequency_tuples.append((collocate, O11, R1, C1, N))

We now define a function to calculate MI

In [60]:
def MI(O11, R1, C1, N):
    # Calculate expected
    E11 = (R1*C1)/N
    # Calculate mutual information to 2 decimal places (actual value not very important)
    mu = round(np.log2(O11/E11), 2)
    return mu

And for each tuple in ```frequency_tuples```, we can calculate the MI score.

This ```for``` loop returns a list of tuples called mi_results, which shows the collocate, how often it occurs alongside our keyword, and the strenght of association between the keyword and collocate (MI).

In [61]:
# list of outputs
mi_results = []

# each group of values in frequency_tuples
for collocate, O11, R1, C1, N in frequency_tuples:
    # Calculate MI using our function
    mutual_information = MI(O11, R1, C1, N)
    # append this to a list
    mi_results.append((collocate, O11, mutual_information))

In [62]:
mi_results[:20]

[('revenge', 1, 5.64),
 ('bright', 1, 3.59),
 ('beautiful', 1, 3.75),
 ('any', 1, 0.38),
 ('these', 1, 1.31),
 ('hated', 1, 6.87),
 ('hate', 1, 6.45),
 ('words', 3, 3.75),
 ('jealousy', 1, 5.28),
 ('in', 10, 0.21),
 ('cannot', 1, 3.36),
 ('very', 1, -0.19),
 ('commonly', 1, 7.45),
 ('neighbor', 1, 5.64),
 ('miss', 4, 1.87),
 ('sending', 1, 6.45),
 ('when', 1, -1.33),
 ('deeper', 3, 7.45),
 ('do', 1, -0.39),
 ('father', 1, 2.34)]

## Cleaner tabular output with Pandas

We can then read this list of tuples into ```pandas``` to allows us to filter and explore the results more easily.

In [63]:
import pandas as pd

In [64]:
dataframe = pd.DataFrame(mi_results, columns =["collocate", "freq", "MI"])

Here we order on two columns, showing the results in descending order for the column ```MI``` and descending order of ```freq``` for collocates with the same MI score.

In [66]:
dataframe.sort_values(["MI", "freq"], ascending=False)[:20]

,collocate,freq,MI
104,wounds,2,8.45
120,reputed,1,8.45
144,favors,2,8.45
40,charity,1,8.45
206,rigidity,1,8.45
17,deeper,3,7.45
71,dire,1,7.45
163,disinterestedness,1,7.45
59,rear,1,7.45
41,generosity,1,7.45
